In [2]:
%load_ext lab_black

In [9]:
!pip install google-search-results

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32077 sha256=91caa5892a429078e732114e7780c7454d4d00f741eb0604b5fccf4d93a8876c
  Stored in directory: c:\users\child\appdata\local\pip\cache\wheels\29\75\71\9bf68178a74593837f73b6e9d9a070d45d308bddfd2e95290a
Successfully built google-search-results


In [7]:
import os

with open("../API_KEY", "r") as f:
    os.environ["OPENAI_API_KEY"] = f.read()
with open("../SERPAPI_KEY", "r") as f:
    os.environ["SERPAPI_API_KEY"] = f.read()

In [4]:
# Import things that are needed generically
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [5]:
llm = ChatOpenAI(temperature=0)

In [11]:
# Load the tool configs that are needed.
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
]

In [12]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    question: str = Field()


tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput
        # coroutine= ... <- you can specify an async method if desired as well
    )
)

In [13]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [14]:
# agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")
agent.run(
    "What is the temperature today in Vancouver? What is that temperature raised to the power 2.75?"
)



> Entering new AgentExecutor chain...
I need to find out the current temperature in Vancouver and then raise it to the power of 2.75. 
Action: Search 
Action Input: "Vancouver temperature today" 
Observation: Current ConditionsObserved at: Vancouver Int'l Airport 11:00 AM PDT Sunday 21 May 2023 ; Condition: Mostly Cloudy; Pressure: 102.1 kPa ; Temperature: 15.6° C · Dew ...
Thought:I have the current temperature in Vancouver, now I need to raise it to the power of 2.75. 
Action: Calculator 
Action Input: 15.6^2.75 

> Entering new LLMMathChain chain...
15.6^2.75

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


```text
15.6**2.75
```
...numexpr.evaluate("15.6**2.75")...

Answer: 1910.2607197321036
> Finished chain.

Observation: Answer: 1910.2607197321036
Thought:I now know the final answer. 
Final Answer: The temperature in Vancouver today is 15.6°C and when raised to the power of 2.75, it is 1910.2607197321036.

> Finished chain.


'The temperature in Vancouver today is 15.6°C and when raised to the power of 2.75, it is 1910.2607197321036.'